In [39]:
import gymnasium as gym
import numpy as np
import os
import torch
import matplotlib.pyplot as plt

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import SubprocVecEnv, VecNormalize
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback, CallbackList
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor


In [40]:

# ✅ Usa SubprocVecEnv per sfruttare il multiprocessing (più veloce di DummyVecEnv)
NUM_ENVS = 4  # Numero di ambienti paralleli per accelerare il training

# ✅ Definiamo la funzione per creare un ambiente vettorializzato
def make_env(healthy_z_lower=0.2746892357443735, healthy_z_upper= 0.9845961239589529,contact_force_min= -0.7057855820088252,contact_force_max=0.6829751913715009):
    return Monitor(gym.make("Ant-v5",
                            reset_noise_scale=0.15280089381590506,
                            forward_reward_weight=0.8100861873330247,
                            ctrl_cost_weight=0.7787466257062721,
                            healthy_reward=1.0,
                            contact_cost_weight=5e-4,
                            healthy_z_range=(healthy_z_lower, healthy_z_upper),
                            contact_force_range=(contact_force_min, contact_force_max),
                            render_mode='none'))

# ✅ Creiamo gli ambienti paralleli
env = SubprocVecEnv([make_env for _ in range(NUM_ENVS)])
env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

/Users/fabiodigregorio/Desktop/campus bio iscrizione/ Magistrale/Merone/RL/Reinforcement_Learning_Ant_MuJoCu/venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:734: UserWarning: WARN: The environment is being initialised with render_mode='none' that is not in the possible render_modes (['human', 'rgb_array', 'depth_array']).
  logger.warn(
/Users/fabiodigregorio/Desktop/campus bio iscrizione/ Magistrale/Merone/RL/Reinforcement_Learning_Ant_MuJoCu/venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:734: UserWarning: WARN: The environment is being initialised with render_mode='none' that is not in the possible render_modes (['human', 'rgb_array', 'depth_array']).
  logger.warn(
/Users/fabiodigregorio/Desktop/campus bio iscrizione/ Magistrale/Merone/RL/Reinforcement_Learning_Ant_MuJoCu/venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:734: UserWarning: WARN: The environment is being initialised with render_mode='none' that is not in the possible r

In [41]:
# ✅ Parametri del modello (puoi ottimizzarli con Optuna)
model_params = {
    "policy": "MlpPolicy",
    "env": env,
    "learning_rate": 1.023387964377036e-05,  # Usa Optuna per trovare il migliore
    "n_steps": 4096,
    "batch_size": 256,
    "n_epochs": 10,
    "gamma": 0.992330458473981,
    "gae_lambda": 0.8064387416284831,
    "clip_range": 0.14933184841818672,
    "ent_coef": 0.017031658560933347,
    "verbose": 1,
    "tensorboard_log": "./ppo_Ant_tensorboard/",
    "device": "mps"  # Usa GPU se disponibile
}

# ✅ Definiamo i callback per salvataggio e valutazione
eval_env = SubprocVecEnv([make_env for _ in range(NUM_ENVS)])
eval_env = VecNormalize(eval_env, norm_obs=True, norm_reward=True, clip_obs=10.)

eval_callback = EvalCallback(eval_env, best_model_save_path="./logs/best_model",
                             log_path="./logs/", eval_freq=5000, deterministic=True, render=False)

checkpoint_callback = CheckpointCallback(save_freq=10000, save_path="./logs/checkpoints/",
                                         name_prefix="ppo_ant_checkpoint")

/Users/fabiodigregorio/Desktop/campus bio iscrizione/ Magistrale/Merone/RL/Reinforcement_Learning_Ant_MuJoCu/venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:734: UserWarning: WARN: The environment is being initialised with render_mode='none' that is not in the possible render_modes (['human', 'rgb_array', 'depth_array']).
  logger.warn(
/Users/fabiodigregorio/Desktop/campus bio iscrizione/ Magistrale/Merone/RL/Reinforcement_Learning_Ant_MuJoCu/venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:734: UserWarning: WARN: The environment is being initialised with render_mode='none' that is not in the possible render_modes (['human', 'rgb_array', 'depth_array']).
  logger.warn(
/Users/fabiodigregorio/Desktop/campus bio iscrizione/ Magistrale/Merone/RL/Reinforcement_Learning_Ant_MuJoCu/venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:734: UserWarning: WARN: The environment is being initialised with render_mode='none' that is not in the possible r

In [42]:
# ✅ Training del modello
model = PPO(**model_params)
model.learn(total_timesteps=100_000, callback=CallbackList([eval_callback, checkpoint_callback]))

# ✅ Salvataggio del modello e della normalizzazione
model.save("ppo_Ant_model")
env.save("vecnormalize_Ant.pkl")

# ✅ Funzione di valutazione migliorata
def evaluate_agent(model, env, episodes=100):
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=episodes, deterministic=True)
    print(f"Mean Reward: {mean_reward:.2f} ± {std_reward:.2f}")
    return mean_reward, std_reward

# ✅ Valutiamo il modello addestrato
mean_reward_trained, std_reward_trained = evaluate_agent(model, env, episodes=200)

/Users/fabiodigregorio/Desktop/campus bio iscrizione/ Magistrale/Merone/RL/Reinforcement_Learning_Ant_MuJoCu/venv/lib/python3.10/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


Using mps device
Logging to ./ppo_Ant_tensorboard/PPO_2


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 45.6     |
|    ep_rew_mean     | -102     |
| time/              |          |
|    fps             | 1267     |
|    iterations      | 1        |
|    time_elapsed    | 12       |
|    total_timesteps | 16384    |
---------------------------------
Eval num_timesteps=20000, episode_reward=1000.60 +/- 3.87
Episode length: 1000.00 +/- 0.00
-------------------------------------------
| eval/                   |               |
|    mean_ep_length       | 1e+03         |
|    mean_reward          | 1e+03         |
| time/                   |               |
|    total_timesteps      | 20000         |
| train/                  |               |
|    approx_kl            | 0.00015621135 |
|    clip_fraction        | 0             |
|    clip_range           | 0.149         |
|    entropy_loss         | -11.4         |
|    explained_variance   | -2.89         |
|    learning_rate        | 1.02e-05     

In [43]:
#5. Salviamo il modello
model.save("ppo_Ant_model")
env.save("vecnormalize_Ant.pkl")  # salviamo anche i parametri di normalizzazione
# ✅ Disattiva la normalizzazione della reward per la valutazione corretta
env.training = False
env.norm_reward = False  

In [44]:
# 200-400 episodi sono adeguati 
def evaluate_policy(env, policy, episodes=500):
    """
    Valuta una policy addestrata su un ambiente dato.

    Parametri:
    - env: L'ambiente di simulazione.
    - policy: La policy addestrata da valutare.
    - episodes: Numero di episodi da eseguire per la valutazione.

    Ritorna:
    - La ricompensa media e la deviazione standard delle ricompense ottenute.
    """
    total_rewards = []
    for _ in range(episodes):
        obs = env.reset()  # Reset dell'ambiente per iniziare un nuovo episodio
        done = False
        total_reward = 0
        while not np.all(done):
            action, _ = policy.predict(obs)  # Predice l'azione da eseguire
            obs, reward, done, _ = env.step(action)  # Esegue l'azione e ottiene il feedback dall'ambiente
            total_reward += reward  # Accumula la ricompensa ottenuta
        total_rewards.append(total_reward)  # Aggiunge la ricompensa totale dell'episodio alla lista
    return np.mean(total_rewards), np.std(total_rewards)  # Calcola e ritorna la media e la deviazione standard delle ricompense

# 200-400 episodi sono adeguati 
def evaluate_random_policy(env, episodes=500):
    """
    Valuta una policy casuale su un ambiente dato.

    Parametri:
    - env: L'ambiente di simulazione.
    - episodes: Numero di episodi da eseguire per la valutazione.

    Ritorna:
    - La ricompensa media e la deviazione standard delle ricompense ottenute.
    """
    total_rewards = []
    for _ in range(episodes):
        obs = env.reset()  # Reset dell'ambiente per iniziare un nuovo episodio
        done = False
        total_reward = 0
        while not np.all(done):
            action = env.action_space.sample()  # Genera un'azione casuale
            obs, reward, done, _ = env.step(np.array([action]))  # Esegue l'azione e ottiene il feedback dall'ambiente
            total_reward += reward  # Accumula la ricompensa ottenuta
        total_rewards.append(total_reward)  # Aggiunge la ricompensa totale dell'episodio alla lista
    return np.mean(total_rewards), np.std(total_rewards)  # Calcola e ritorna la media e la deviazione standard delle ricompense

In [45]:
# Valutazione dopo l'addestramento
mean_reward_trained, std_reward_trained = evaluate_policy(env, model)  # Valuta la policy addestrata
mean_reward_random, std_reward_random = evaluate_random_policy(env)  # Valuta la policy casuale

# Stampa dei risultati
print(f"Trained Policy: Mean Reward: {mean_reward_trained}, Std: {std_reward_trained}")
print(f"Random Policy: Mean Reward: {mean_reward_random}, Std: {std_reward_random}")

# Creazione del grafico di confronto
labels = ['Random Policy', 'Trained Policy']
means = [mean_reward_random, mean_reward_trained]
stds = [std_reward_random, std_reward_trained]

plt.figure(figsize=(8, 5))
plt.bar(labels, means, yerr=stds, capsize=10, color=['skyblue', 'lightgreen'])
plt.ylabel('Mean Episodic Reward')
plt.title('Policy Comparison')
plt.show()

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()